# Lecon 4 : L'attention

## L'ingredient secret des GPT

Imagine que tu lis la phrase : "Le **chat** noir dort sur le **canape**."

Si on te demande "Ou dort le chat ?", ton cerveau fait automatiquement
le lien entre "dort", "chat" et "canape" -- meme si ces mots ne sont
pas cote a cote.

C'est exactement ce que fait le **mecanisme d'attention** :
il permet au modele de regarder **n'importe quel** element du passe,
pas seulement les derniers.

## Comment ca marche ?

Pour chaque lettre, le modele se pose 3 questions :

1. **Query (Q)** : "Qu'est-ce que je cherche ?" (ce que cette lettre a besoin de savoir)
2. **Key (K)** : "Qu'est-ce que j'offre ?" (ce que cette lettre peut apporter)
3. **Value (V)** : "Quelle info je transmets ?" (l'information reelle)

L'attention = comparer chaque Query avec toutes les Keys pour trouver
les lettres les plus utiles, puis collecter leurs Values.

In [ ]:
import math
import random

random.seed(42)

# Simulons un mot simple
mot = "chat"
print(f"Mot : '{mot}'")
print(f"Positions : {list(enumerate(mot))}")
print()

# Chaque lettre a un embedding (simplifie a 4 dimensions)
DIM = 4
emb = {
    'c': [1.0, 0.0, 0.5, -0.3],
    'h': [0.2, 0.8, -0.1, 0.5],
    'a': [0.5, 0.3, 0.9, 0.1],
    't': [-0.3, 0.6, 0.2, 0.8],
}

for c, v in emb.items():
    print(f"  '{c}' -> {v}")

In [ ]:
# Pour simplifier, on utilise les embeddings directement comme Q, K, V
# (en vrai, il y a des matrices de transformation)

def produit_scalaire(a, b):
    """Mesure la similarite entre deux vecteurs."""
    return sum(x * y for x, y in zip(a, b))

def softmax(scores):
    """Transforme les scores en probabilites."""
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]

# Calculons l'attention pour la lettre 't' (derniere position)
# Question : quelles lettres precedentes sont importantes pour predire
# ce qui vient apres 't' dans 'chat' ?

query = emb['t']  # Ce que 't' cherche

# Comparer avec toutes les lettres precedentes (y compris elle-meme)
scores = []
for c in mot:
    key = emb[c]  # Ce que chaque lettre offre
    score = produit_scalaire(query, key) / math.sqrt(DIM)
    scores.append(score)

print("Scores d'attention pour 't' :")
for c, s in zip(mot, scores):
    print(f"  '{c}' : {s:.2f}")

# Transformer en probabilites
poids_attention = softmax(scores)
print()
print("Poids d'attention (apres softmax) :")
for c, w in zip(mot, poids_attention):
    barre = '#' * int(w * 30)
    print(f"  '{c}' : {w:.1%} {barre}")

print()
print("Le modele 'regarde' plus les lettres avec un poids eleve !")

In [ ]:
# Collecter l'information (somme ponderee des Values)

resultat = [0.0] * DIM
for c, w in zip(mot, poids_attention):
    value = emb[c]
    for d in range(DIM):
        resultat[d] += w * value[d]

print("Vecteur de sortie de l'attention :")
print(f"  {[f'{x:.2f}' for x in resultat]}")
print()
print("Ce vecteur combine l'information de toutes les lettres,")
print("en donnant plus de poids aux lettres les plus pertinentes.")
print()
print("C'est cette information qui sera utilisee pour predire")
print("la lettre suivante !")

## Attention causale : pas de triche !

Regle importante : quand le modele predit la lettre suivante,
il **ne peut pas regarder le futur** -- seulement le passe.

```
Pour predire apres 'c' : peut regarder [c]
Pour predire apres 'h' : peut regarder [c, h]
Pour predire apres 'a' : peut regarder [c, h, a]
Pour predire apres 't' : peut regarder [c, h, a, t]
```

On appelle ca le **masque causal**. C'est ce qui fait de GPT un modele
**auto-regressif** : il genere un token a la fois, de gauche a droite.

In [ ]:
# Visualisons le masque causal
print("Masque causal pour 'chat' :")
print()
print("          c    h    a    t")
for i, c in enumerate(mot):
    row = ""
    for j in range(len(mot)):
        if j <= i:
            row += "  OK "
        else:
            row += "  -- "
    print(f"  {c} : {row}")

print()
print("OK = peut regarder, -- = interdit (c'est le futur)")

## Multi-tetes : regarder de plusieurs facons

En pratique, on utilise **plusieurs tetes d'attention** en parallele.
Chaque tete apprend a chercher un type d'information different :

- Tete 1 : quelles lettres forment des syllabes ensemble ?
- Tete 2 : quelle est la voyelle la plus recente ?
- Tete 3 : est-ce que c'est un debut ou une fin de mot ?

Les resultats de toutes les tetes sont combines pour une prediction finale.

Dans microgpt.py de Karpathy, il y a **4 tetes** d'attention.

## Resume visuel

```
Lettres d'entree :  [c] [h] [a] [t]
                     |   |   |   |
                     v   v   v   v
Embeddings :        [---] [---] [---] [---]
                     |   |   |   |
                     v   v   v   v
Attention :         Chaque lettre regarde les precedentes
                    et collecte l'info importante
                     |   |   |   |
                     v   v   v   v
Prediction :        Quelle est la prochaine lettre ?
```

## Ce qu'on a appris

- L'**attention** permet au modele de regarder toutes les lettres precedentes, pas juste la derniere
- Ca fonctionne avec **Q** (je cherche), **K** (j'offre), **V** (mon info)
- Le **masque causal** empeche de tricher en regardant le futur
- Plusieurs **tetes** d'attention regardent des choses differentes en parallele

### Derniere etape

On a maintenant toutes les pieces du puzzle. Dans la prochaine lecon,
on assemble tout pour construire un vrai mini-LLM !

---
*Prochaine lecon : [05 - Mon premier LLM](05_mon_premier_llm.ipynb)*